In [1]:
# coding=UTF-8

import time
from datetime import datetime
import requests
import sys
from bs4 import BeautifulSoup
requests.packages.urllib3.disable_warnings()
import pandas as pd
import re
import os
import json

In [2]:
Board = 'graduate'
page = 506#(往回爬幾頁)
start_index=1105
file_name=str(Board)+str((start_index-page)+1)+'-'+str(start_index)

In [3]:
payload={
'from':'/bbs/'+ Board +'/index.html',
'yes':'yes' 
}

data_list=[]
fdata=[]

def getPageNumber(content) :
    startIndex = content.find('index')
    endIndex = content.find('.html')
    pageNumber = content[startIndex+5 : endIndex]
    return pageNumber

# python PttCrawler.py Gossiping 2 
if __name__ == "__main__":
    Board = Board
    ParsingPage =int(page)
    print('Start parsing ['+ Board +']....')
    start_time = time.time()
    rs = requests.session()
    #八卦版18禁
    res = rs.post('https://www.ptt.cc/ask/over18',verify = False, data = payload)
    res = rs.get('https://www.ptt.cc/bbs/'+ Board +'/index.html',verify = False)
    soup = BeautifulSoup(res.text,'html.parser')
    ALLpageURL = soup.select('.btn.wide')[1]['href']
    ALLpage = int(getPageNumber(ALLpageURL)) + 1
    print('Total pages:', ALLpage)
    URLlist=[]
    fileName='PttData-'+ Board + '-' + datetime.now().strftime('%Y%m%d%H%M%S')+'.txt'

    #得到每頁的所有文章
    for index in range(start_index, start_index-int(ParsingPage), -1):
        print('index',index)
        url = 'https://www.ptt.cc/bbs/'+ Board +'/index'+ str(index) +'.html'
        res = rs.get(url, verify = False)
        soup = BeautifulSoup(res.text,'html.parser')
        UrlPer = []
        for entry in soup.select('.r-ent'):
            atag = entry.select('.title')[0].find('a') 
            if atag != None:
                URL = atag['href']          
                UrlPer.append('https://www.ptt.cc' + URL)   
        #需要反轉,因為網頁版最下面才是最新的文章
        for URL in reversed(UrlPer):
            URLlist.append(URL)   
    
    strNext = u"\n\n\n\n***************下一篇***************\n\n\n\n\n";
    content = ''
    #得到每篇文章的內容
    for URL in URLlist:
        res = rs.get(URL, verify = False)
        soup = BeautifulSoup(res.text, 'html.parser')
        main_content = soup.find(id="main-content")
        #data = soup.select('.bbs-screen.bbs-content')
        #metas = soup.select(
        #content += (data + strNext)
        try:
            metas = main_content.select('div.article-metaline')
        except:
            main_content=None
            #print('main_error')
            
        author = ''
        title = ''
        date = ''
        #print('URL',URL)
        if metas:
            try:
                author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
                title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
                date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
            except:
                metas=None
                #print('meta_list_error')
        # remove meta nodes
            try:
                for meta in metas:
                    meta.extract()
                for meta in main_content.select('div.article-metaline-right'):
                    meta.extract()
            except:
                main_content=None
                #print('meta_error')
            
                
            time.sleep(0.05) 
        # remove and keep push nodes
        try:
            pushes = main_content.find_all('div', class_='push')
            for push in pushes:
                push.extract()
        except:
            main_content=None
            #print('push_error')

        try:
            ip = main_content.find(text=re.compile(u'※ 發信站:'))
            ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
        except:
            ip = "None"
        time.sleep(0.05) 

        # 移除 '※ 發信站:' (starts with u'\u203b'), '◆ From:' (starts with u'\u25c6'), 空行及多餘空白
        # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
        try:
            filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
            expr = re.compile(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]')
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            filtered = [_f for _f in filtered if _f]  # remove empty strings
            #filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
            content = ' '.join(filtered)
            content = re.sub(r'(\s)+', ' ', content)
            # print 'content', content
            #print('content',content)
        except:
            main_content=None
            #print('filtered_error')
            
        # push messages
        p, b, n = 0, 0, 0
        messages = []
        for push in pushes:
            if not push.find('span', 'push-tag'):
                continue
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            # if find is None: find().strings -> list -> ' '.join; else the current way
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')
            messages.append( {'push_tag': push_tag, 'push_userid': push_userid, 'push_content': push_content, 'push_ipdatetime': push_ipdatetime} )
            if push_tag == u'推':
                p += 1
            elif push_tag == u'噓':
                b += 1
            else:
                n += 1
                
        # count: 推噓文相抵後的數量; all: 推文總數
        message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, "neutral": n}

        data={'author':author,
              'title':title,
              'date':date,
              'content': content,
              'message_count': message_count,
              'messages': messages}
        
        js=json.dumps(data,sort_keys=True, ensure_ascii=False)
        df=pd.read_json(js, typ="index")
        fdata.append(df)

    df=pd.DataFrame(fdata)
    df.to_csv(str(file_name)+'.csv', index=False, encoding='utf-8-sig')
    
        
        
       
    #with open(fileName,'wb') as f:
#        f.write( content.encode('utf8') )  
     
    print(u'====================END====================')
    print(u'execution time:' + str(time.time() - start_time)+'s')
    


Start parsing [graduate]....
Total pages: 1958
index 1105
index 1104
index 1103
index 1102
index 1101
index 1100
index 1099
index 1098
index 1097
index 1096
index 1095
index 1094
index 1093
index 1092
index 1091
index 1090
index 1089
index 1088
index 1087
index 1086
index 1085
index 1084
index 1083
index 1082
index 1081
index 1080
index 1079
index 1078
index 1077
index 1076
index 1075
index 1074
index 1073
index 1072
index 1071
index 1070
index 1069
index 1068
index 1067
index 1066
index 1065
index 1064
index 1063
index 1062
index 1061
index 1060
index 1059
index 1058
index 1057
index 1056
index 1055
index 1054
index 1053
index 1052
index 1051
index 1050
index 1049
index 1048
index 1047
index 1046
index 1045
index 1044
index 1043
index 1042
index 1041
index 1040
index 1039
index 1038
index 1037
index 1036
index 1035
index 1034
index 1033
index 1032
index 1031
index 1030
index 1029
index 1028
index 1027
index 1026
index 1025
index 1024
index 1023
index 1022
index 1021
index 1020
index 1

In [5]:
df.head(10)

,author,content,date,message_count,messages,title
0,luke7459 (TsingHua),因為有學弟妹在拜託了 想說那乾脆一起開給大家 我目前構想會幫大家準備幾個部分 [我心得文版上...,Sun May 11 21:55:02 2014,"{'all': 5, 'boo': 0, 'count': 5, 'neutral': 0,...","[{'push_content': '大推!!!第一個報名!!!', 'push_ipdat...",[閒聊] 科法所準備班
1,MisterYun (氣質小甜心),各位版上專家好 小弟在畢業後有計畫想考台北大學法律系研究所 但因為非本科系不知道如何準備起 ...,Sun May 11 12:28:15 2014,"{'all': 19, 'boo': 0, 'count': 6, 'neutral': 1...","[{'push_content': '如果法律專業組，主要看的是你的backgro', 'p...",[請益] 台北大學法律系相關問題請教
2,rex589032555 (Brisk),不好意思打擾各位，請益一下 彰師的積體電路，有位老師的專長是 類比IC 與 RF IC 感覺...,Sun May 11 01:38:37 2014,"{'all': 3, 'boo': 0, 'count': 2, 'neutral': 1,...","[{'push_content': '找大咖老師 早點進去學', 'push_ipdatet...",[請益] 彰師_積體電路 與 海大_通訊導航
3,biker0710 (biker),小弟我在上個月已經先報到中央，結果這個月中興通知我備取上了 請問中央生科分子與細胞生物組 V...,Sun May 11 01:29:38 2014,"{'all': 0, 'boo': 0, 'count': 0, 'neutral': 0,...",[],[請益] 中興生物科技所和中央分子與細胞組比較
4,xhow4dox (xhow4dox),有鑑於板上每年都有迷途羔羊向眾板友詢問xx vs yy而引發多場血腥戰爭... 小的不才，在...,Sun May 11 00:13:01 2014,"{'all': 30, 'boo': 0, 'count': 14, 'neutral': ...","[{'push_content': '感謝原po把所有會考慮的因素都列出來並', 'push...",[心得] 以量化後的分數做理性思考
5,ape0808 (ape猿),小弟最近才備上 希望能準時7.8.9月畢業 我已經有看過系網上教授們的研究領域， 我選了幾位...,Sat May 10 23:36:34 2014,"{'all': 9, 'boo': 0, 'count': 5, 'neutral': 4,...","[{'push_content': '3 早9晚9 要會打程式。', 'push_ipdat...",[請益] 南交找教授
6,gary87004,首先要感謝板上各個高手的心得文 讓我在考前有些方向去準備 所以想要分享一下自己的心得給各位 ...,Sat May 10 21:51:49 2014,"{'all': 12, 'boo': 0, 'count': 12, 'neutral': ...","[{'push_content': '強者我同學', 'push_ipdatetime': ...",[心得] 台大電信乙組正取心得
7,tenjow (天照),hcsh14th/NUKThor1211 警告一次 HORSEwinner/SmithEE4...,Sat May 10 16:40:55 2014,"{'all': 420, 'boo': 7, 'count': 131, 'neutral'...","[{'push_content': '報名上了還再酸= =當初幹嘛報...', 'push_...",[公告] HORSEwinner/SmithEE4598水桶 hcsh14th/NUKTho...
8,gp3u04g6 (明夜),各位板友好~ 這裡有份學術問卷需要請大家幫個忙~ a)問卷全名：電子商務網站中的商品文案設計...,Sat May 10 16:09:23 2014,"{'all': 13, 'boo': 0, 'count': 13, 'neutral': ...","[{'push_content': 'done', 'push_ipdatetime': '...",[問卷] 電商網站中的商品文案設計(100p+抽獎)
9,programer (ken),各位版上前輩們好 最近才收到通知，需要開始找教授 可是周圍認識的朋友很少人清楚 來請問各位前...,Sat May 10 14:08:15 2014,"{'all': 3, 'boo': 0, 'count': 2, 'neutral': 1,...","[{'push_content': '要不要海大版問看看', 'push_ipdatetim...",[請益] 海大食科找教授
